In [14]:
import cv2
import torch
import numpy as np
import os
import sys
sys.path.append('./SuperGluePretrainedNetwork')
from models.matching import Matching
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# 📐 ChArUco 보드 정의
squares_x, squares_y = 13, 9
square_length = 0.02  # meters
marker_length = 0.01  # meters
dict_type = cv2.aruco.DICT_5X5_100
aruco_dict = cv2.aruco.getPredefinedDictionary(dict_type)
board = cv2.aruco.CharucoBoard(
    (squares_x, squares_y),
    square_length,
    marker_length,
    aruco_dict
)

# 📥 이미지 로딩
def load_images(folder):
    return sorted([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.jpeg')])

# 🔧 SuperGlue 매칭기 초기화
matching = Matching({
    'superpoint': {'nms_radius': 4, 'keypoint_threshold': 0.005, 'max_keypoints': 1024},
    'superglue': {'weights': 'indoor'},
    'sinkhorn_iterations': 20,
    'match_threshold': 0.2,
}).eval().to(device)

def estimate_pose(charuco_corners, charuco_ids, board, K, dist):
    """
    각 charuco 코너의 ID에 맞춰 해당하는 3D 보드 좌표를 수동 계산한 뒤,
    이미지 코너들과 매칭시켜 solvePnP 수행
    """
    obj_pts = []
    img_pts = []

    squares_x, squares_y = board.getChessboardSize()
    square_length = board.getSquareLength()

    for i in range(len(charuco_ids)):
        id = int(charuco_ids[i][0])
        row = id // (squares_x - 1)
        col = id % (squares_x - 1)

        # 3D board 상 위치 (Z=0 평면)
        obj_pts.append([col * square_length, row * square_length, 0.0])
        img_pts.append(charuco_corners[i][0])  # 2D 이미지 좌표

    if len(obj_pts) < 4:
        return False, None, None

    obj_pts = np.array(obj_pts, dtype=np.float32)
    img_pts = np.array(img_pts, dtype=np.float32)

    success, rvec, tvec = cv2.solvePnP(obj_pts, img_pts, K, dist)
    return success, rvec, tvec

# 📊 시각화
def plot_camera_views(poses):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for idx, (R, t) in enumerate(poses):
        cam_center = -R.T @ t
        ax.scatter(cam_center[0], cam_center[1], cam_center[2], c='r')
        ax.text(cam_center[0], cam_center[1], cam_center[2], f'Cam {idx}', size=8)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.set_zlabel("Z (m)")
    ax.set_title("Camera Poses")
    plt.show()

# === 실행 ===
image_paths = load_images('./camera_images3')
K = np.array([[800, 0, 640], [0, 800, 360], [0, 0, 1]], dtype=np.float32)
dist = np.zeros((5, 1))

poses = []

# 🔹 첫 이미지 → 기준
ref_img = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
ref_tensor = torch.from_numpy(ref_img / 255.).float()[None, None].to(device)

# 기준 이미지에서 ChArUco 추출
corners0, ids0, _ = cv2.aruco.detectMarkers(ref_img, aruco_dict)
if ids0 is not None and len(corners0) > 0:
    _, charuco_corners0, charuco_ids0 = cv2.aruco.interpolateCornersCharuco(corners0, ids0, ref_img, board)
    if charuco_corners0 is not None and charuco_ids0 is not None and len(charuco_ids0) >= 4:
        success, rvec0, tvec0 = estimate_pose(charuco_corners0, charuco_ids0, board, K, dist)
        if success:
            R0, _ = cv2.Rodrigues(rvec0)
            poses.append((R0, tvec0))

# 🔄 나머지 이미지들
for i in tqdm(range(1, len(image_paths))):
    img = cv2.imread(image_paths[i], cv2.IMREAD_GRAYSCALE)
    tensor = torch.from_numpy(img / 255.).float()[None, None].to(device)

    # SuperGlue 매칭 (기준 ref vs 현재 이미지)
    inp = {'image0': ref_tensor, 'image1': tensor}
    pred = matching(inp)

    # 결과 추출
    kpts0 = pred['keypoints0'][0].cpu().numpy()
    kpts1 = pred['keypoints1'][0].cpu().numpy()
    matches = pred['matches0'][0].cpu().numpy()

    valid = matches > -1
    matched_kpts0 = kpts0[valid]
    matched_kpts1 = kpts1[matches[valid]]

    # 현재 이미지에서 ChArUco 추출
    corners, ids, _ = cv2.aruco.detectMarkers(img, aruco_dict)
    if ids is not None and len(corners) > 0:
        _, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, img, board)
        if charuco_corners is not None and charuco_ids is not None and len(charuco_ids) >= 4:
            success, rvec, tvec = estimate_pose(charuco_corners, charuco_ids, board, K, dist)
            if success:
                R, _ = cv2.Rodrigues(rvec)
                poses.append((R, tvec))

# 📊 모든 pose 시각화
plot_camera_views(poses)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


  0%|          | 0/14 [00:04<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 17.38 GB, other allocations: 146.70 MB, max allowed: 18.13 GB). Tried to allocate 2.13 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).